## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bin6_2 = pd.read_csv("data/"+'answer_2.csv', index_col=0)
print(bin6_2.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
bin6_2.reset_index(inplace=True, drop=True)

print('reset_index 완료')
print('input data shape')
bin6_2.head()

(21033568, 2)
reset_index 완료
input data shape


,bin,label
0,47,0
1,108,0
2,105,0
3,98,0
4,47,0


In [2]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 
# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(bin6_2['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(bin6_2['label'].value_counts())

256
0    21007896
1       25672
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [3]:
idx_bin = bin6_2[bin6_2['label']==1].index  # 407, 474 ...
ls_bin = list(idx_bin)

# 최종 뽑을 행에 대한 index
ls_idx_bin = []

# n byte 자르기 방식
left_idx, right_idx = 0, 32 # 3개씩

# n byte 자르기
for k in range(left_idx, right_idx):
    ls_idx_bin.extend(list(idx_bin + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx_bin.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx_bin = list(filter(lambda x: x<len(bin6_2), ls_idx_bin))
print(len(ls_idx_bin))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub_bin = len(ls_idx_bin)%(right_idx)
print('나머지', sub_bin)

ls_idx_bin = ls_idx_bin[:len(ls_idx_bin)-sub_bin]
print('최종 길이', len(ls_idx_bin))

print('bin6_2', len(ls_idx_bin))

# loc 로 수정필요
bin6_2_Ngram = bin6_2.loc[ls_idx_bin,:].copy()

821504
나머지 0
최종 길이 821504
bin6_2 821504


## (4) false data 만들기

In [4]:
# false data 만들기 - False 데이터 랜덤 생성

# 목표치
goal_bin = len(bin6_2_Ngram)/right_idx
count_bin = 0

print(goal_bin)

# 최종 데이터 Frame
d_bin = pd.DataFrame(columns = bin6_2.columns)

binutils_df = []
# goal 에 도달할 때까지
while True:
    if (count_bin == goal_bin):
            break
    # 진행상황 살펴보기 위함
            
    # 랜덤 N 바이트씩 뽑음
    # random index
    random_idx_bin = np.random.randint(len(bin6_2)-right_idx)

    if count_bin % 1000==0:
        print(count_bin, end=' ')
        print(random_idx_bin)

    df_bin = bin6_2[random_idx_bin : random_idx_bin + right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df_bin['label'] and count_bin < goal_bin:
        binutils_df.append(df_bin)
        count_bin+=1

print('완료')
print(len(binutils_df))

25672.0
0 12775243
1000 18322669
2000 3498504
3000 17832316
4000 9857233
5000 15859659
6000 7275246
7000 2757030
8000 11707505
9000 8168429
10000 2460153
11000 10931056
12000 20978162
13000 8146384
14000 9287154
15000 14231361
16000 8774132
17000 19378870
18000 673061
19000 279871
20000 17855193
21000 12647334
22000 8625968
23000 10702996
24000 2258967
25000 5434327
완료
25672


In [5]:
# True data와 False Data 같은지 체크
print(len(binutils_df))
print(bin6_2['label'].value_counts()[1])

25672
25672


## (5) False Data + True Data 합치기

In [6]:
f_data = pd.concat(binutils_df)
final = pd.concat([f_data, bin6_2_Ngram])
final.shape

(1643008, 2)

## (6) one hot encoding

In [7]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
bc6_2_onehot_Ngram = pd.get_dummies(final['bin'])
bc6_2_onehot_Ngram = pd.concat([final['label'], bc6_2_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(bc6_2_onehot_Ngram.shape)

원핫인코딩완료
(1643008, 257)


In [8]:
# 훈련 데이터, 훈련 라벨
x_bc6_2 = bc6_2_onehot_Ngram.iloc[:,1:].to_numpy()
y_bc6_2 = bc6_2_onehot_Ngram['label'].to_numpy()
print(x_bc6_2.shape, x_bc6_2.shape)

x_bc6_2 = x_bc6_2.reshape(-1, right_idx, x_bc6_2.shape[1])
y_bc6_2 = y_bc6_2.reshape(-1, right_idx, 1)

print(x_bc6_2.shape, y_bc6_2.shape)

(1643008, 256) (1643008, 256)
(51344, 32, 256) (51344, 32, 1)


In [9]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bc6_2.shape[0])

x_bc6_2 = x_bc6_2[p]
y_bc6_2 = y_bc6_2[p]

print(x_bc6_2.shape, y_bc6_2.shape)

(51344, 32, 256) (51344, 32, 1)


## (7) 모델

In [10]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(48, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습 - 10 KFold

In [11]:
# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=True, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_bc6_2, y_bc6_2):
    model1 = models.Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model1.summary()
    print('======Training stage======')
    model1.fit(x_bc6_2[train],
               y_bc6_2[train],
               epochs = 10,
               batch_size = 128,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_bc6_2[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_bc6_2[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 46209 samples
Epoch 1/10
46209/46209 [==============================] - 10s 218us/sample - loss: 0.0447 - accuracy: 0.9879
Epoch 2/10
46209/46209 [==============================] - 3s 62us/sample - loss: 0.0060 - accuracy: 0.9986
Epoch 3/10
46209/46209 [==============================] - 3s 62us/sample - loss: 0.0020 - accuracy: 0.9994
Epoch 4/10
4

accuracy_score 0.99972005842259
recall_score 0.9939148073022313
precision_score 0.9905660377358491
f1_score 0.9922375970300371
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 46209 samples
Epoch 1/10
46209/46209 [==============================] - 6s 129us/sample - loss: 4.9587e-04 - accuracy: 0.9999
Epoch 2/10
46209/46209 [==============================] - 3s 62us/sample - loss: 4.0815e-04 - 

accuracy_score 0.9998235150925024
recall_score 0.9951640759930915
precision_score 0.9948204419889503
f1_score 0.9949922293213607
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 46209 samples
Epoch 1/10
46209/46209 [==============================] - 5s 117us/sample - loss: 2.0232e-04 - accuracy: 0.9999
Epoch 2/10
46209/46209 [==============================] - 3s 63us/sample - loss: 1.4445e-04 

accuracy_score 0.9999452288218111
recall_score 0.9996564754379939
precision_score 0.9972583961617546
f1_score 0.9984559958826557
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 46209 samples
Epoch 1/10
46209/46209 [==============================] - 5s 115us/sample - loss: 4.5414e-05 - accuracy: 1.0000
Epoch 2/10
46209/46209 [==============================] - 3s 62us/sample - loss: 2.5220e-05 

accuracy_score 0.9999878286270691
recall_score 0.9996436208125445
precision_score 0.9996436208125445
f1_score 0.9996436208125445
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 46210 samples
Epoch 1/10
46210/46210 [==============================] - 6s 140us/sample - loss: 2.5817e-05 - accuracy: 1.0000
Epoch 2/10
46210/46210 [==============================] - 3s 65us/sample - loss: 1.8974e-05 

...

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 46210 samples
Epoch 1/10
46210/46210 [==============================] - 7s 157us/sample - loss: 1.2305e-06 - accuracy: 1.0000
Epoch 2/10
46210/46210 [==============================] - 3s 63us/sample - loss: 4.1855e-06 - accuracy: 1.0000
Epoch 3/10
46210/46210 [=================

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 46210 samples
Epoch 1/10
46210/46210 [==============================] - 6s 134us/sample - loss: 8.4090e-07 - accuracy: 1.0000
Epoch 2/10
46210/46210 [==============================] - 3s 70us/sample - loss: 1.1369e-07 - accuracy: 1.0000
Epoch 3/10
46210/46210 [=================

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 46210 samples
Epoch 1/10
46210/46210 [==============================] - 6s 130us/sample - loss: 3.1508e-09 - accuracy: 1.0000
Epoch 2/10
46210/46210 [==============================] - 3s 71us/sample - loss: 2.7844e-09 - accuracy: 1.0000
Epoch 3/10
46210/46210 [=================

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 46210 samples
Epoch 1/10
46210/46210 [==============================] - 8s 180us/sample - loss: 1.9987e-09 - accuracy: 1.0000
Epoch 2/10
46210/46210 [==============================] - 3s 69us/sample - loss: 1.9265e-09 - accuracy: 1.0000
Epoch 3/10
46210/46210 [=================

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0

K-fold cross validation Accuracy: [0.99972005842259, 0.9998235150925024, 0.9999452288218111, 0.9999878286270691, 0.9999939131281652, 1.0, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation Recall: [0.9939148073022313, 0.9951640759930915, 0.9996564754379939, 0.9996436208125445, 0.9996596324029953, 1.0, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation Precision: [0.9905660377358491, 0.9948204419889503, 0.9972583961617546, 0.9996436208125445, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation F1-Score: [0.9922375970300371, 0.9949922293213607, 0.9984559958826557, 0.9996436208125445, 0.9998297872340426, 1.0, 1.0, 1.0, 1.0, 1.0]


## (9) 평가지표

In [12]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9999470544092137
10-Fold Cross_validation. Recall : 0.9988038611948855
10-Fold Cross_validation. Precision : 0.9982288496699099
10-Fold Cross_validation. F1-Score : 0.998515923028064


In [15]:
model1.save('gcc6_bin_s32_h48_o2.h5')
print('save 완료')

save 완료


In [16]:
from tensorflow.keras.models import load_model
model_path = 'gcc6_bin_s32_h48_o2.h5'

model = load_model(model_path)

# 12. 교차검증결과 predict - 검증셋들
# predict 값
k_pr = model.predict(x_bc6_2)

# 테스트 predict 결과들 비교 (평가지표 보기위함)
pred = np.round(np.array(k_pr).flatten().tolist())
y_test = np.array(y_bc6_2).flatten().tolist()
print(len(pred), len(y_test))

# 13. 평가지표들 출력
## 평가지표들
k_accuracy = float(accuracy_score(y_test, pred))
k_recall =  float(recall_score(y_test, pred))
k_precision = float(precision_score(y_test, pred))
k_f1_score = float(f1_score(y_test, pred))
#k_cm = float(confusion_matrix(y_test, pred))

print('accuracy_score', k_accuracy)
print('recall_score', k_recall)
print('precision_score', k_precision)
print('f1_score', k_f1_score)
#print('\nconfusion_matrix\n', k_cm)

1643008 1643008
accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
